In [5]:
from ngsolve import *
from netgen.geom2d import SplineGeometry

from netgen.occ import *
import netgen.meshing as ngm
import numpy as np
from ngsolve.webgui import *
from netgen.geom2d import unit_square



# generate a triangular mesh of mesh-size 0.2
mesh = Mesh(unit_square.GenerateMesh(maxh=0.01))
max_iter = 20
#phi = CF(sqrt(0.25-(x-0.5)**2-(y-0.5)**2) if (x-0.5)**2+(y-0.5)**2 < 0.25 else 0)
phi = IfPos(0.04-(x-0.5)**2-(y-0.5)**2, sqrt(0.04-(x-0.5)**2-(y-0.5)**2), 0)
alpha = 0.5
# psi initial guess
psik = CF(0)
for k in range(max_iter):
    print("Iteration of proximal galerkin: ", k)
    fesu = H1(mesh, order=12, dirichlet="bottom|right|top|left")
    fesphi = L2(mesh, order=10)
    fes = fesu*fesphi
    u, varphi = fes.TrialFunction()
    v, psi = fes.TestFunction()
    a = BilinearForm(fes)
    a += alpha*grad(u)*grad(v)*dx + varphi*v*dx
    a += u*psi*dx - exp(varphi)*psi*dx

    l = LinearForm(fes)
    l += (0+psik)*v*dx
    l += phi*psi*dx

    a.Assemble()
    l.Assemble()
    # No lifting given g = 0 on \partial\Omegagfu = GridFunction(fes)
    gfu = GridFunction(fes)
    res = gfu.vec.CreateVector()
    w = gfu.vec.CreateVector()
    history = GridFunction(fes, multidim = 0)
    # here we may need to add another loop
    maxiter = 15
    print("Initializing Newton")
    for iter in range(maxiter):
        
        # Prints before the iteration: number of it, residual, energy
        
        a.Apply(gfu.vec, res)
        a.AssembleLinearization(gfu.vec)
        inv = a.mat.Inverse(freedofs = fes.FreeDofs())        
        w.data = 0.5 * inv * res
        gfu.vec.data -= w
        history.AddMultiDimComponent(gfu.vec)
        if sqrt(abs(InnerProduct(w,res))) < 1e-6:
            print("Converged")
            break
    
    print("Exiting newton with:", InnerProduct(w,res))
    psik = gfu.components[1]
    Draw(exp(gfu.components[1])+ phi , mesh, "psi")
    #Draw(gfu.components[0], mesh, "u")
tilde_u = phi + exp(gfu.components[1])  


    



Iteration of proximal galerkin:  0
Initializing Newton


NgException: PardisoInverse: Setup and Factorization failed.

In [ ]:
phi = IfPos(0.04-(x-0.5)**2-(y-0.5)**2, sqrt(0.04-(x-0.5)**2-(y-0.5)**2), 0)
Draw(phi, mesh, "tilde_u")

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene

In [ ]:
# solve the simple laplacian example
fes = H1(mesh, order=2, dirichlet="bottom|right")
u, v = fes.TnT()
a = BilinearForm(fes)
a += grad(u)*grad(v)*dx
l = LinearForm(fes)
l += 0*v*dx
a.Assemble()
l.Assemble()
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(fes.FreeDofs()) * l.vec
Draw(gfu, mesh, "u")


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene